## Data Cleaning

* Data cleaning is data set specific but there are some common problems
    * Missing values
    * Duplicates
    * Reporting or collection bias or drift
    * Reporting or collection errors
* Data cleaning is more effective when it is combined with visualization and some preliminary modeling
* Decribe all the steps you take to load and clean the data so that you and others can repeat the process

### In Class Exercise

Use your functions or the ones below to load the train data into a dataframe and combine all the narrative fields into one variable: Narrative. Optionally, remove the previous narrative columns.

In [1]:
# Packages for loading, cleaning, visualization, and analysis

# Data
import pandas as pd
import numpy as np
import scipy as sp
import os
import string as st


In [2]:
# Function to get the files from a directory

def getallfiles(directory, extension = ".txt"):
    '''Get all files in directory with the specified extension
        and put them into a list.
        The default extension is txt. The directory parameter is the path to 
        the directory containing the files.'''
    filenames = os.listdir(directory)
    myfiles = []
    for e in filenames:
        if e.endswith(extension):
            myfiles.append(os.path.realpath(e))
    return myfiles




In [3]:
def createlist(directory, extension = ".txt"):
    '''Put all files in the specified directory
    with the chosen extension (txt is the default) 
    into a datafame'''
    os.chdir(directory)
    files = getallfiles(directory)
    filelist = []
    for i,file in enumerate(files):
        filelist.append(pd.read_csv(os.path.realpath(file), low_memory = False, encoding = "ISO-8859-1"))
    return(filelist)


In [7]:
# Create the data frame of all accidents

path = "/Users/Pan/Google Drive/Data Science/DS6001"
acts = createlist(path)
accidents_df = pd.concat(acts,ignore_index=True)
accidents_df.shape

(51623, 153)

In [8]:
# Join the narrative and put them in a list

def join_narratives(DF):
    '''With the input of the accident dataframe
    merge the narrative columns into a single narrative
    and return a list of these single narratives for each
    accident report in the dataframe. '''
    narrlist = []
    for i in range(0,15):
        a = str(i+1)
        narrlist.append('NARR'+ a)
    RailNarr = DF.loc[:, narrlist]
    Narratives = []
    for i, _ in enumerate(RailNarr["NARR1"]):
        NarrativeList = RailNarr.iloc[i]
        Anarrative = ""
        for narr in NarrativeList:
            if pd.isnull(narr):
                break
            else:
                Anarrative += str(narr)
    Narratives.append(Anarrative)

In [9]:
narrative_list = join_narratives(accidents_df)
accidents_df["Narrative"] = narrative_list

#Check by looking at narrative 3
accidents_df["Narrative"][3]

In [10]:
# Drop the old narrative columns
narrlist = []
for i in range(0,15):
    a = str(i+1)
    narrlist.append('NARR'+a)
accidents_df.drop(narrlist,axis =1,inplace=True)

## Duplicates

- Why should we remove duplicates?
- How should we remove them?

In [11]:
# Look at 3 approaches to removing duplicates

#Removing duplicates
accidents_clean_df = accidents_df.drop_duplicates(['YEAR','DAY','MONTH','TIMEHR'])
print(accidents_clean_df.shape)

#Using AMPM
accidents_clean_df = accidents_df.drop_duplicates(['YEAR','DAY','MONTH','TIMEHR','AMPM'])
print(accidents_clean_df.shape)

# Using the FRA FAQ (look at the source of the data)
accidents_clean_df = accidents_df[(accidents_df['JOINTCD']==1)&(accidents_df['TYPE']!=7)]
print(accidents_clean_df.shape)

(30677, 139)
(35453, 139)
(38167, 139)


## Missing Values

There are essentially 3 ways to handle missing values:

    1. Remove the columns (variables) with missing values
    2. Remove the row (observations) with missing values
    3. Impute the missing values

The choice of which of these to use depends on the problem and the data. If a variable does not seem important to the problem or if it has many missing values, then eliminating it is reasonable. Similarly if the observation appears to not represent the data or if it has many missing values, then eliminating it seems reasonable.
Imputation

Imputation can be done in may ways. The most common are the following:

    1. Replace the missing value with the mean
    2. Replace the missing value with the median
    3. Replace the missing value with the mode
    4. Use k-nn to
        1. Replace the missing value with the mean
        2. Replace the missing value with the median
        3. Replace the missing value with the mode
 
Options 1 and 2 can only be used with numeric or quantitative variables.

### In Class Exercise

1. Find out how many missing values we have by column (variable).
2. What technique or techniques should we use to handle them?
3. What methods from pandas or sklearn can we use for imputation? Why or how would we use them?

In [12]:
#How many missing values?

accidents_clean_df.isnull().sum()

ACCAUSE          0
ACCDMG           0
ACCTRK          55
ACCTRKCL        69
ADJUNCT1     32271
ADJUNCT2     37540
ADJUNCT3     37938
ALCOHOL      26857
AMPM             4
AMTRAK       36762
BRAKEMEN      2903
CABOOSE1         0
CABOOSE2         0
CARNBR1       1396
CARNBR2      33412
CARS             0
CARSDMG          0
CARSHZD          0
CASINJ           0
CASINJRR         0
CASKLD           0
CASKLDRR         0
CAUSE            0
CAUSE2       33474
CDTRHR        9012
CDTRMIN      12848
CNTYCD          11
CONDUCTR      1472
COUNTY           0
CauseCat     38167
             ...  
SUBDIV       28208
TEMP             0
TIMEHR           0
TIMEMIN          0
TONS             0
TOTINJ           0
TOTKLD           0
TRKCLAS        216
TRKDMG           0
TRKDNSTY     22061
TRKNAME        230
TRNDIR        1744
TRNNBR        4188
TRNSPD           0
TYPE             0
TYPEQ         1384
TYPRR           16
TYPSPD        1423
TYPTRK         194
VISIBLTY         0
WEATHER          0
YEAR        

In [13]:
# Remove variables with more than 1.5K missing (about 4%)

accidents_clean_df = accidents_clean_df.dropna(axis=1,thresh = (38167-1500))
accidents_clean_df.isnull().sum()

ACCAUSE        0
ACCDMG         0
ACCTRK        55
ACCTRKCL      69
AMPM           4
CABOOSE1       0
CABOOSE2       0
CARNBR1     1396
CARS           0
CARSDMG        0
CARSHZD        0
CASINJ         0
CASINJRR       0
CASKLD         0
CASKLDRR       0
CAUSE          0
CNTYCD        11
CONDUCTR    1472
COUNTY         0
DAY            0
DUMMY3         0
EMPTYF1        0
EMPTYF2        0
EMPTYP1        0
EMPTYP2        0
ENGRS       1158
EQATT       1419
EQPDMG         0
EVACUATE       0
HEADEND1       0
            ... 
RMAN1          0
RMAN2          0
RR3            0
RRCAR1      1405
RREM1          0
RREM2          0
RREMPINJ       0
RREMPKLD       0
STATE          0
STATION        1
STCNTY        11
TEMP           0
TIMEHR         0
TIMEMIN        0
TONS           0
TOTINJ         0
TOTKLD         0
TRKCLAS      216
TRKDMG         0
TRKNAME      230
TRNSPD         0
TYPE           0
TYPEQ       1384
TYPRR         16
TYPSPD      1423
TYPTRK       194
VISIBLTY       0
WEATHER       

In [19]:
# This class imputes the missing values as
# (1) most frequent if the variable is categorical 
# (2) mean if the variable is real (floating point)
# (3) median if the variable is an integer 

# Here is a class that will provide imputation
# This is an extension by D.Brown to sveitser, 2014 https://stackoverflow.com/users/469992/sveitser

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.
        
        Columns of dtype floating point are imputed with the mean.

        Columns of other types are imputed with median of the column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') 
                               else X[c].mean() if X[c].dtype == np.dtype('f')
                                else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


In [22]:
accidents_clean_df = DataFrameImputer().fit_transform(accidents_clean_df)


ACCAUSE     0
ACCDMG      0
ACCTRK      0
ACCTRKCL    0
AMPM        0
CABOOSE1    0
CABOOSE2    0
CARNBR1     0
CARS        0
CARSDMG     0
CARSHZD     0
CASINJ      0
CASINJRR    0
CASKLD      0
CASKLDRR    0
CAUSE       0
CNTYCD      0
CONDUCTR    0
COUNTY      0
DAY         0
DUMMY3      0
EMPTYF1     0
EMPTYF2     0
EMPTYP1     0
EMPTYP2     0
ENGRS       0
EQATT       0
EQPDMG      0
EVACUATE    0
HEADEND1    0
           ..
RMAN1       0
RMAN2       0
RR3         0
RRCAR1      0
RREM1       0
RREM2       0
RREMPINJ    0
RREMPKLD    0
STATE       0
STATION     0
STCNTY      0
TEMP        0
TIMEHR      0
TIMEMIN     0
TONS        0
TOTINJ      0
TOTKLD      0
TRKCLAS     0
TRKDMG      0
TRKNAME     0
TRNSPD      0
TYPE        0
TYPEQ       0
TYPRR       0
TYPSPD      0
TYPTRK      0
VISIBLTY    0
WEATHER     0
YEAR        0
YEAR4       0
Length: 87, dtype: int64

## Categorical Variables

Categorical variables must be treated differently from quantitative variables. Their effects on the modeling
and machine learning depend on correctly coding them for the analysis. 

Categorical variables can be entered in many diffent ways. Data scientists should inspect the coding 
of these variables to insure correctness, approrpriateness for modeling, and easy interpretation.

### In Class Exercise

1. Look at the data types for the variables.
2. Which variables are categorical? Look at some of their value_counts()
3. Which variables are categorical but are coded as integers? Which variables are integers but coded as objects?
4. Replace integer values for TYPE with the text labels. Repeat for one other variable.

In [14]:
#1 The data types
accidents_clean_df.dtypes

ACCAUSE      object
ACCDMG      float64
ACCTRK      float64
ACCTRKCL     object
AMPM         object
CABOOSE1    float64
CABOOSE2    float64
CARNBR1      object
CARS        float64
CARSDMG     float64
CARSHZD     float64
CASINJ      float64
CASINJRR    float64
CASKLD      float64
CASKLDRR    float64
CAUSE        object
CNTYCD      float64
CONDUCTR     object
COUNTY       object
DAY           int64
DUMMY3        int64
EMPTYF1     float64
EMPTYF2     float64
EMPTYP1     float64
EMPTYP2     float64
ENGRS        object
EQATT        object
EQPDMG      float64
EVACUATE    float64
HEADEND1    float64
             ...   
RMAN1       float64
RMAN2       float64
RR3          object
RRCAR1       object
RREM1       float64
RREM2       float64
RREMPINJ    float64
RREMPKLD    float64
STATE         int64
STATION      object
STCNTY       object
TEMP        float64
TIMEHR      float64
TIMEMIN     float64
TONS        float64
TOTINJ      float64
TOTKLD      float64
TRKCLAS      object
TRKDMG      float64


In [37]:
#2.Which variables are categorical? Look at some of their value_counts()
cat_list = accidents_clean_df.columns[accidents_clean_df.dtypes == 'object']

for name in cat_list:
    print('Variable Name:'+name)
    print(accidents_clean_df[name].value_counts())
    print("")

Variable Name:ACCAUSE
T110    2350
H702    2068
H306    1478
T314    1025
H307     994
T220     946
M405     723
T207     699
H704     612
H503     548
H607     526
T221     525
H302     517
M404     478
M411     457
H018     457
T111     429
T109     424
H303     422
T210     411
H318     409
H020     403
E53C     403
T202     393
M599     393
H312     390
T102     372
T311     362
M101     359
T001     350
        ... 
H821       2
E30L       2
E81C       2
E02L       2
H899       2
E67L       2
S008       1
E33L       1
S001       1
E52L       1
E04L       1
H515       1
E31L       1
E24L       1
E61L       1
H304       1
E36L       1
E63L       1
E19C       1
E25L       1
E4TL       1
E40L       1
E35L       1
E03L       1
E01C       1
H99D       1
H517       1
M303       1
S101       1
E34L       1
Name: ACCAUSE, Length: 359, dtype: int64

Variable Name:ACCTRKCL
1    24957
4     3844
2     3675
3     2609
X     1742
5      961
6      177
7      136
8       55
9       11
Name: ACCT

In [41]:
#3. Which variables are categorical but are coded as integers?  
int_list = accidents_clean_df.columns[accidents_clean_df.dtypes == 'int']
for name in int_list:  
    if accidents_clean_df[name].nunique()<20:
        print('Variable Name:'+name)
        print(accidents_clean_df[name].value_counts())
        print("")
#ANS: IYR, IYR3, YEAR, YEAR4,MONTH, REGION, STATE, VISIBLTY, WEATHER, YEAR, YEAR4

Variable Name:DUMMY3
0    38167
Name: DUMMY3, dtype: int64

Variable Name:IMO
7     3452
3     3322
8     3315
1     3305
6     3283
12    3218
2     3146
5     3128
4     3057
9     3042
10    3029
11    2870
Name: IMO, dtype: int64

Variable Name:IMO3
7     3452
3     3322
8     3315
1     3305
6     3283
12    3218
2     3146
5     3128
4     3057
9     3042
10    3029
11    2870
Name: IMO3, dtype: int64

Variable Name:IYR
4     3385
5     3266
1     3023
3     3019
6     2998
2     2738
7     2693
8     2435
11    1996
15    1906
9     1902
10    1884
13    1800
14    1775
12    1712
16    1635
Name: IYR, dtype: int64

Variable Name:IYR3
4     3385
5     3266
1     3023
3     3019
6     2998
2     2738
7     2693
8     2435
11    1996
15    1906
9     1902
10    1884
13    1800
14    1775
12    1712
16    1635
Name: IYR3, dtype: int64

Variable Name:JOINTCD
1    38167
Name: JOINTCD, dtype: int64

Variable Name:MONTH
7     3452
3     3322
8     3315
1     3305
6     3283
12    3218


Index(['DAY', 'DUMMY3', 'HIGHSPD', 'IMO', 'IMO3', 'IYR', 'IYR3', 'JOINTCD',
       'MONTH', 'POSITON1', 'POSITON2', 'REGION', 'STATE', 'TRNSPD',
       'VISIBLTY', 'WEATHER', 'YEAR', 'YEAR4'],
      dtype='object')

In [43]:
#3. Which variables are integers but coded as objects?
#based on qn2
#ANS: CARNBR1, CONDUCTR, ENGRS


In [16]:
#4. Replace values TYPE
accidents_clean_df['TYPE']=accidents_clean_df['TYPE'].replace(range(1,14),
                                    ["Defailment","Headon","Rearend","Side","Raking",
                                    "BrokenTrain","Hwy-Rail","GradeX","Obstruction",
                                    "Explosive","Fire","Other","SeeNarrative"])
accidents_clean_df['TYPE'].value_counts()

Defailment      27429
Other            4321
SeeNarrative     2127
Side             1603
Obstruction       991
Raking            720
Fire              454
Rearend           298
Headon            125
BrokenTrain        80
Explosive          14
GradeX              5
Name: TYPE, dtype: int64

In [17]:
# Map for TYPEQ
# Taken from Rail Equipment Accident/ Incident Report Form

map_typeq = {1:"Freight",2:"PassengerPull",3:"Commuter",4:"Work",5:"Single",
            6:"CutofCars",7:"Yard",8:"Light",9:"Maint",'A':"Maint of Way",
            '1':"Freight",'2':"PassengerPull",'3':'Commuter','4':"Work",'5':"Single",
            '6':"CutofCars",'7':"Yard",'8':"Light",'9':"Maint",'B':"PassengerPush",'C':"CommuterPush",
            'D':"EMU",'E':"DMU"}
accidents_clean_df['TYPEQ'] = accidents_clean_df['TYPEQ'].map(map_typeq)
accidents_clean_df['TYPEQ'].value_counts()

Freight          16604
Yard             12910
CutofCars         1950
Light             1719
PassengerPull     1188
Single            1048
Commuter           363
Maint of Way       311
Maint              254
Work               252
EMU                101
CommuterPush        38
PassengerPush       28
DMU                 17
Name: TYPEQ, dtype: int64